In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score , precision_score

In [2]:
import os, sys, gc, warnings, random, datetime, math

def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)

train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
data = pd.concat([train, val, test])

In [3]:
def make_copy(data):
    data_ = data.copy()
    return data_

In [4]:
lws = [i for i in list(data.columns) if i.startswith('gas')]
data.drop(lws,1,inplace=True)

In [5]:
bins = np.linspace(0,2500, 1000)
train['fr_wthr_fclt_dstnc'] = np.digitize(train['fr_wthr_fclt_dstnc'], bins)

In [6]:
len(np.digitize(data['fr_wthr_fclt_dstnc'], bins))

69054

In [7]:
len(data)

69054

In [8]:
a = train.groupby('fr_wthr_fclt_dstnc').sum() /train.groupby('fr_wthr_fclt_dstnc').count()

In [9]:
[1,2,3,4][1]

2

In [10]:
data['name_1'] = data['emd_nm'].str[5:8]

In [12]:
data['name_4'] = data['name_1'].str[2]

In [14]:
data['name_2'] = data['emd_nm'].str[::-1].str[:].str[2]+data['emd_nm'].str[::-1].str[:].str[1]+data['emd_nm'].str[::-1].str[:].str[0]

In [16]:
data['name_3'] =data['emd_nm'].str[::-1].str[:].str[0]

In [ ]:
data['name_1'] = data['emd_nm'].str[5:8]

data['name_4'] = data['name_1'].str[2]

data['name_2'] = data['emd_nm'].str[::-1].str[:].str[2]+data['emd_nm'].str[::-1].str[:].str[1]+data['emd_nm'].str[::-1].str[:].str[0]

data['name_3'] =data['emd_nm'].str[::-1].str[:].str[0]

In [39]:
rm_cols = ['id']

In [54]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1_score', f1_score(y_true, y_hat), True

def lgbmgo(data_, train, test, rm_cols):

    train_ = data_[:len(train) + 2500]
    val_ = data_[len(train)+2500:-len(test)]
    test_ = data_[-len(test):]
    dataset = [train_, val_, test_]

    for df in dataset:
        categorical_cols1 = df.select_dtypes(['object']).columns
        for col in categorical_cols1:
            df[col] = pd.Categorical(df[col]).codes
    train_.drop(rm_cols,1,inplace=True)
    val_.drop(rm_cols,1,inplace=True)
    test_.drop(rm_cols,1,inplace=True)
    X_train = train_.drop('fr_yn', 1)
    y_train = train_['fr_yn']
    X_val = val_.drop('fr_yn', 1)
    y_val = val_['fr_yn']
    X_test = test_.drop(columns = ['fr_yn'])
    
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape)
    #모델 생성 및 학습
    params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'metric' :'Recall',
                    'seed' :SEED,
                }
    
    lgb_train = lgb.Dataset(X_train ,label= y_train)
    lgb_eval = lgb.Dataset(X_val, label= y_val)
    y_prob = np.zeros((X_test.shape[0]))

    model = lgb.train(params, lgb_train,valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=300, verbose_eval=100)
    
    y_pred = model.predict(X_val)[:]
    val_pred = pd.DataFrame(y_pred)
    val_pred[0] = val_pred[0]>0.5
    val_pred[0] = val_pred[0].map({False:0,True:1})

    score = precision_score(y_val, val_pred[0])
    score2 = accuracy_score(y_val, val_pred[0])

    score3 = recall_score(y_val, val_pred[0])
    score4 = f1_score(y_val, val_pred[0])

    print('AUC_score :', score,'| Acc_score :', score2,'| recall_score :', score3,'| f1_score :', score4)
    gc.collect()

## EDA

In [55]:
# train.head()

In [56]:
data_ = data.copy()
no_fe = lgbmgo(data_, train, test,rm_cols)

(61699, 184) (4398, 184) (61699,) (4398,) (2957, 184)


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

## FE

In [ ]:
data['dt_of_fr'] = pd.to_datetime(data['dt_of_fr'])
data['DT_hour'] = (data['dt_of_fr'].dt.hour)
#data['DT_Y'] = (data['dt_of_fr'].dt.year)
# data['DT_day_week'] = (data['dt_of_fr'].dt.dayofweek).astype(np.int8)
# data['DT_day_month'] = (data['dt_of_fr'].dt.day).astype(np.int8)
data['DT_M'] = (data['dt_of_fr'].dt.month)
# data = data.drop('dt_of_fr',axis = 1)
rm_cols += ['dt_of_fr']

In [ ]:
data['dt_of_athrztn'].fillna(100, inplace=True)
data['dt_of_athrztn_year'] = data['dt_of_athrztn'].astype(str).str[:4].astype(int)
bins = np.linspace(1600,2019, 20)
data['dt_of_athrztn_year2'] = np.digitize(data['dt_of_athrztn_year'], bins)
#data['diff_athrztn_frY'] = data['DT_Y']-data['dt_of_athrztn_year']
#data = data.drop(['dt_of_athrztn','DT_Y'], axis = 1)
rm_cols += ['dt_of_athrztn']

In [52]:
rm_cols

['id', 'dt_of_fr', 'dt_of_athrztn']

In [53]:
data_ = data.copy()
fe_time = lgbmgo(data_, train, test,rm_cols)


(61699, 184) (4398, 184) (61699,) (4398,) (2957, 184)


ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [349]:
data['ttl_sum'] = data['ttl_grnd_flr'] + data['ttl_dwn_flr']
rm_cols += ['ttl_grnd_flr','ttl_dwn_flr']

In [350]:
data_ = data.copy()
fe_time = lgbmgo(data_, train, test,rm_cols)


(59199, 181) (6898, 181) (59199,) (6898,) (2957, 181)
Training until validation scores don't improve for 300 rounds.
[100]	training's f1_score: 0.271522	valid_1's f1_score: 0.421693
[200]	training's f1_score: 0.450091	valid_1's f1_score: 0.515378
[300]	training's f1_score: 0.515971	valid_1's f1_score: 0.518919
[400]	training's f1_score: 0.544029	valid_1's f1_score: 0.518575
[500]	training's f1_score: 0.56917	valid_1's f1_score: 0.514979
Early stopping, best iteration is:
[225]	training's f1_score: 0.470953	valid_1's f1_score: 0.52589
AUC_score : 0.5416666666666666 | Acc_score : 0.8300956799072194 | recall_score : 0.5110062893081762 | f1_score : 0.5258899676375405


In [351]:
bins = np.linspace(0,800000, 1000)
data['ttl_ar'] = np.digitize(data['ttl_ar'], bins)
#bins = np.linspace(0,1000, 100)
#data['lnd_ar'] = np.digitize(data['lnd_ar'], bins)

In [352]:
data_ = data.copy()
fe_time = lgbmgo(data_, train, test,rm_cols)


(59199, 181) (6898, 181) (59199,) (6898,) (2957, 181)
Training until validation scores don't improve for 300 rounds.
[100]	training's f1_score: 0.257108	valid_1's f1_score: 0.40853
[200]	training's f1_score: 0.453129	valid_1's f1_score: 0.509379
[300]	training's f1_score: 0.516176	valid_1's f1_score: 0.517665
[400]	training's f1_score: 0.547056	valid_1's f1_score: 0.525009
[500]	training's f1_score: 0.571654	valid_1's f1_score: 0.523702
[600]	training's f1_score: 0.591544	valid_1's f1_score: 0.525765
[700]	training's f1_score: 0.60698	valid_1's f1_score: 0.521674
[800]	training's f1_score: 0.621268	valid_1's f1_score: 0.520466
Early stopping, best iteration is:
[577]	training's f1_score: 0.586977	valid_1's f1_score: 0.52785
AUC_score : 0.5032074126870991 | Acc_score : 0.8169034502754422 | recall_score : 0.5550314465408805 | f1_score : 0.5278504672897197


In [238]:
train['train_yn'] = 0

train['train_yn'][train['prcpttn'] != 0] = 1

In [239]:
data_ = data.copy()
fe_time = lgbmgo(data_, train, test,rm_cols)


(59199, 178) (6898, 178) (59199,) (6898,) (2957, 178)
Training until validation scores don't improve for 300 rounds.
[100]	training's f1_score: 0.259259	valid_1's f1_score: 0.406285
[200]	training's f1_score: 0.452791	valid_1's f1_score: 0.514168
[300]	training's f1_score: 0.519758	valid_1's f1_score: 0.516546
[400]	training's f1_score: 0.543304	valid_1's f1_score: 0.517717
[500]	training's f1_score: 0.569952	valid_1's f1_score: 0.518135
[600]	training's f1_score: 0.591791	valid_1's f1_score: 0.519795
[700]	training's f1_score: 0.606805	valid_1's f1_score: 0.520818
[800]	training's f1_score: 0.617554	valid_1's f1_score: 0.520879
[900]	training's f1_score: 0.629384	valid_1's f1_score: 0.522852
[1000]	training's f1_score: 0.643075	valid_1's f1_score: 0.52016
[1100]	training's f1_score: 0.652452	valid_1's f1_score: 0.521488
Early stopping, best iteration is:
[862]	training's f1_score: 0.624499	valid_1's f1_score: 0.523775
AUC_score : 0.4897400820793434 | Acc_score : 0.811249637576109 | re

In [ ]:
lws = [i for i in list(df_all.columns) if i.startswith('lw')]
df_all.drop(lws,1,inplace=True)

In [ ]:
data['danger1'] = data['no_tbc_zn_dstnc'] - data['tbc_rtl_str_dstnc']

In [ ]:
data['prevent1'] = data['cctv_in_100m'] + data['fr_wthr_fclt_in_100m']

In [ ]:
# rm_cols.remove('DT_Y')

In [ ]:
rm_cols

In [ ]:
# bins = np.linspace(-19175,9795, 300)
# data['danger1'] = np.digitize(data['danger1'], bins)

In [ ]:
# bins = np.linspace(0,16164, 160)
# data['no_tbc_zn_dstnc'] = np.digitize(data['no_tbc_zn_dstnc'], bins)

In [ ]:
# bins = np.linspace(0,24000, 240)
# data['tbc_rtl_str_dstnc'] = np.digitize(data['tbc_rtl_str_dstnc'], bins)

In [ ]:
data_ = data.copy()
fe_time_ath_ttl_dan_pre = lgbmgo(data_, train, test,rm_cols)

In [ ]:
# data['cctv_in_100m'][data['cctv_in_100m'] >= 1] = 1

In [ ]:
# data['fr_wthr_fclt_in_100m'][data['fr_wthr_fclt_in_100m'] >= 1] = 1

In [ ]:
# data_ = data.copy()
# fe_time_ath_ttl_dan_pre = lgbmgo(data_, train, test)

In [327]:
test.head()

,ahsm_dstnc,bldng_ar,bldng_ar_prc,bldng_archtctr,bldng_cnt,bldng_cnt_in_50m,bldng_us,bldng_us_clssfctn,blk_dngrs_thng_mnfctr_yn,cctv_dstnc,...,tmprtr,trgt_crtr,ttl_ar,ttl_dwn_flr,ttl_grnd_flr,us_yn,wnd_drctn,wnd_spd,name_1,name_2
0,32295,0.00,NaN,-1,3,0,-1,-1,-1,148,...,6.6,-1,0.00,NaN,NaN,-1,290.0,4.5,0,1
1,18413,915.75,NaN,11,2,8,1,1,-1,281,...,13.8,-1,903.75,0.0,1.0,-1,110.0,2.2,0,8
2,4903,5523.87,NaN,-1,5,1,-1,-1,0,353,...,12.9,-1,9888.87,0.0,9.0,0,320.0,2.2,0,15
3,5729,667.20,NaN,-1,8,0,-1,-1,-1,571,...,12.4,-1,914.40,0.0,3.0,-1,140.0,0.6,0,15
4,8205,1050.06,344255.0,-1,7,4,-1,-1,-1,560,...,22.0,-1,1050.33,0.0,3.0,-1,290.0,4.6,0,16


In [27]:
val.head()

,ahsm_dstnc,bldng_ar,bldng_ar_prc,bldng_archtctr,bldng_cnt,bldng_cnt_in_50m,bldng_us,bldng_us_clssfctn,blk_dngrs_thng_mnfctr_yn,cctv_dstnc,...,wnd_drctn,wnd_spd,name_1,name_4,name_2,name_3,DT_hour,DT_Y,DT_M,dt_of_athrztn_year


In [24]:
    train = data[:len(train)+2500]
    val = data[len(train)+2500:-len(test)]
    test = data[-len(test):]
    dataset = [train, val, test]

    for df in dataset:
        categorical_cols1 = df.select_dtypes(['object']).columns
        for col in categorical_cols1:
            df[col] = pd.Categorical(df[col]).codes
    train.drop(rm_cols,1,inplace=True)
    val.drop(rm_cols,1,inplace=True)
    test.drop(rm_cols,1,inplace=True)
    X_train = train.drop('fr_yn', 1)
    y_train = train['fr_yn']
    X_val = val.drop('fr_yn', 1)
    y_val = val['fr_yn']
    X_test = test.drop(columns = ['fr_yn'])
    
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape)
    #모델 생성 및 학습
    params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'metrics' : 'auc',
                    'seed' :SEED,
                }
    
    lgb_train = lgb.Dataset(X_train ,label= y_train)
    lgb_eval = lgb.Dataset(X_val, label= y_val)
    y_prob = np.zeros((X_test.shape[0]))

    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=100, verbose_eval=100)
    
    y_pred = model.predict(X_val)[:]
    val_pred = pd.DataFrame(y_pred)
    val_pred[0] = val_pred[0]>0.5
    val_pred[0] = val_pred[0].map({False:0,True:1})

    score = precision_score(y_val, val_pred[0])
    score2 = accuracy_score(y_val, val_pred[0])

    score3 = recall_score(y_val, val_pred[0])
    score4 = f1_score(y_val, val_pred[0])

    print('precision :', score,'| Acc_score :', score2,'| recall_score :', score3,'| f1_score :', score4)
    gc.collect()

(64199, 184) (0, 184) (64199,) (0,) (2957, 184)


ValueError: Input data must be 2 dimensional and non empty.

In [400]:
y = model.predict(X_test)


In [401]:
val_pred = pd.DataFrame(y)
val_pred[0] = val_pred[0]>0.5
val_pred[0] = val_pred[0].map({False:0,True:1})

In [402]:
sub['fr_yn'] = val_pred[0]
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.to_csv('tmddnjs94_1128_진우_NEW9.csv', index=False)